In [1]:
%cd /content/drive/MyDrive/Fall23/Capstone Research/new_efforts

/content/drive/MyDrive/Fall23/Capstone Research/new_efforts


In [2]:
!pip install torchmetrics

In [3]:
import tensorflow as tf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
import warnings

from torchmetrics.regression import MeanAbsolutePercentageError
from tqdm import tqdm


In [4]:
ds_path = "/content/drive/MyDrive/Fall23/Capstone Research/new_efforts/extracted_15_10_cv/0"
ds_val = "/content/drive/MyDrive/Fall23/Capstone Research/new_efforts/extracted_15_10_cv/0"
ds_train = tf.data.Dataset.load(f"{ds_path}/training", compression="GZIP")
ds_val = tf.data.Dataset.load(f"{ds_path}/validation", compression="GZIP")


In [5]:
import numpy as  np

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# Convert TF dataset to PyTorch dataset
class CustomDataset(Dataset):
    def __init__(self, tf_dataset):
        self.tf_dataset = tf_dataset

    def __len__(self):
        # Calculate the length of the dataset
        # Implement according to the structure of your dataset
        return len(self.tf_dataset)

    def __getitem__(self, idx):
        # Retrieve an item from the dataset
        # Implement according to the structure of your dataset
        return self.tf_dataset[idx]

In [6]:
train_list = []
# X_graphs = []
for item in ds_train:
    # Assuming the elements in the dataset are tensors, convert them into a format compatible with PyTorch
    train_list.append((item[0], item[1].numpy()))

val_list = []
# X_graphs = []
for item in ds_val:
    # Assuming the elements in the dataset are tensors, convert them into a format compatible with PyTorch
    val_list.append((item[0], item[1].numpy()))


In [7]:
min_max_scores_fields = {
    "flow_traffic",
    "flow_packets",
    "flow_packet_size",
    "link_capacity",
}

In [8]:
min_max_scores_fields

{'flow_packet_size', 'flow_packets', 'flow_traffic', 'link_capacity'}

In [9]:
def get_min_max_dict(
    ds: tf.data.Dataset, params, include_y = None
):
    """Get the min and the max-min for different parameters of a dataset. Later used by the models for the min-max normalization.

    Parameters
    ----------
    ds : tf.data.Dataset
        Training dataset where to base the min-max normalization from.

    params : List[str]
        List of strings indicating the parameters to extract the features from.

    include_y : Optional[str], optional
        Indicates if to also extract the features of the output variable.
        Inputs indicate the string key used on the return dict. If None, it is not included.
        By default None.

    Returns
    -------
    Dict[str, Tuple[np.ndarray, np.ndarray]]
        Dictionary containing the values needed for the min-max normalization.
        The first value is the min value of the parameter, and the second is 1 / (max - min).
    """

    # Use first sample to get the shape of the tensors
    iter_ds = iter(ds)
    sample, label = next(iter_ds)
    params_lists = {param: sample[param].numpy() for param in params}
    if include_y:
        params_lists[include_y] = label.numpy()

    # Include the rest of the samples
    for sample, label in iter_ds:
        for param in params:
            params_lists[param] = np.concatenate(
                (params_lists[param], sample[param].numpy()), axis=0
            )
        if include_y:
            params_lists[include_y] = np.concatenate(
                (params_lists[include_y], label.numpy()), axis=0
            )

    scores = dict()
    for param, param_list in params_lists.items():
        min_val = np.min(param_list, axis=0)
        min_max_val = np.max(param_list, axis=0) - min_val
        if min_max_val.size == 1 and min_max_val == 0:
            scores[param] = [min_val, 0]
            print(f"Min-max normalization Warning: {param} has a max-min of 0.")
        elif min_max_val.size > 1 and np.any(min_max_val == 0):
            min_max_val[min_max_val != 0] = 1 / min_max_val[min_max_val != 0]
            scores[param] = [min_val, min_max_val]
            print(
                f"Min-max normalization Warning: Several values of {param} has a max-min of 0."
            )
        else:
            scores[param] = [min_val, 1 / min_max_val]  ## return min and 1/max-min

    return scores


In [10]:
import random
random.shuffle(train_list)
train_dataset = CustomDataset(train_list)
val_dataset = CustomDataset(val_list)


In [11]:
normalizers = get_min_max_dict(ds_train, min_max_scores_fields)

Transformer

In [12]:
import torch
import torch.nn.functional as F


import torch.nn.functional as F
import torch.nn as nn

class PathEmbedding(nn.Module):
    def __init__(self, path_state_dim):
        super(PathEmbedding, self).__init__()
        self.path_state_dim = path_state_dim
        self.flow_embedding = nn.Sequential(
            nn.Linear(7, self.path_state_dim),
            nn.GELU(),
            nn.Linear(self.path_state_dim, self.path_state_dim),
            nn.GELU()
        )

    def forward(self, x):
        return self.flow_embedding(x)

class LinkEmbedding(nn.Module):
    def __init__(self, link_state_dim):
        super(LinkEmbedding, self).__init__()
        self.link_state_dim = link_state_dim
        self.link_embedding = nn.Sequential(
            nn.Linear(2, self.link_state_dim),
            nn.GELU(),
            nn.Linear(self.link_state_dim, self.link_state_dim),
            nn.GELU()
        )

    def forward(self, x):
        return self.link_embedding(x)

class PathReadout(nn.Module):
    def __init__(self, path_state_dim, link_state_dim):
        super(PathReadout, self).__init__()
        self.path_state_dim = path_state_dim
        self.link_state_dim = link_state_dim
        self.readout_path = nn.Sequential(
            nn.Linear(self.path_state_dim, self.link_state_dim // 2),
            nn.GELU(),
            nn.Linear(self.link_state_dim // 2, self.link_state_dim // 4),
            nn.GELU(),
            nn.Linear(self.link_state_dim // 4, 1),
            nn.Softplus()
        )

    def forward(self, x):
        return self.readout_path(x)






In [13]:

class MyCombinedModel(nn.Module):
    def __init__(self, link_state_dim, path_state_dim, num_layers=2):
        super(MyCombinedModel, self).__init__()
        self.link_embed = LinkEmbedding(link_state_dim)  # Assuming LinkEmbedding is defined
        self.flow_embedded = PathEmbedding(path_state_dim)  # Assuming PathEmbedding is defined
        self.reader = PathReadout(path_state_dim, link_state_dim)  # Assuming PathReadout is defined
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=link_state_dim, dim_feedforward=64, batch_first=True, nhead=2)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.link_update = nn.GRUCell(
          link_state_dim, link_state_dim
        )

        self.path_update = nn.GRUCell(
          path_state_dim, path_state_dim
        )
    def forward(self, link_to_path, path_to_link,  link_capacity,  all_flows_used, flow_traffic, link_capacity_orig):

        path_state = self.flow_embedded(all_flows_used)


        ##########################
          ## link to path     ##
        #########################


        all_link_capacities_used = []
        loads = []

        for i in range(len(path_to_link)):

            loads.append(torch.sum(torch.index_select(torch.tensor(flow_traffic), 0, torch.tensor(path_to_link[i][:, 0 ]).type(torch.int64)))/  (link_capacity_orig[i] * 1e9))


        loads_capacities = torch.cat((torch.tensor(loads).unsqueeze(1), torch.tensor(link_capacity)), dim=1)


        ## initialize the hidden state for links
        link_state = self.link_embed(loads_capacities)



        for _ in range(4):

          ##########################
            ## link to path     ##
          #########################
          link_gather  = []
          for link_to_pat in link_to_path:
              link_gather.append(link_state[link_to_pat])
          prev_path_state = path_state

          hx = prev_path_state

          path_state = []
          path_state_sequence = []
          for i, link_stated in enumerate(link_gather):

            hx_i = self.path_update(link_stated, torch.tile(hx[i].unsqueeze(0), (len(link_stated),1)))
            path_state_sequence.append(torch.cat( (prev_path_state[i].unsqueeze(0),  hx_i), dim = 0  ))

            path_state.append(hx_i[-1, :])

          path_state = torch.stack(path_state)



          ##########################
            ## path to link     ##
          #########################

          path_sum = []

          padded_tensor =  torch.nn.utils.rnn.pad_sequence(path_state_sequence, batch_first=True, padding_value=0)



          for path_to_lin in path_to_link:
              path_sum.append(torch.sum(padded_tensor[path_to_lin[:,0], path_to_lin[:,1]], dim=0) )

          path_sum = torch.stack(path_sum)
          link_state = self.link_update(path_sum, link_state)


        capacity_gather = []
        for i, values in enumerate(link_to_path):
          capacity_gather.append(link_capacity_orig[values])

        delays = []

        for i, path_state_seq in enumerate(path_state_sequence):
            occupancy = self.reader(path_state_seq[1:, :])
            delays.append(torch.sum(occupancy[:len(capacity_gather[i])]/torch.tensor(capacity_gather[i]).T[0]).unsqueeze(0))


        return torch.cat(delays)

link_state_dim = 64
path_state_dim = 64

from torch.optim.lr_scheduler import ReduceLROnPlateau
num_layers = 2  # Example value for the number of layers

model = MyCombinedModel(link_state_dim,  path_state_dim, num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# optimizer =  torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
# scheduler = ReduceLROnPlateau(optimizer, patience=10, mode= 'min')
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.00001, max_lr=0.1)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = MeanAbsolutePercentageError()

def train():
  model.train()
  fake_batch_size= 8
  with tqdm(train_dataset, total=len(train_dataset)) as pbar:
    predictions = []
    y_all = []
    loss = 0
    loss = torch.tensor([0.0])

    for i,batch in enumerate(pbar):
        # print(batch)
        # Process the batch as needed
        data, labels = batch  # Modify this according to the structure of your data
        # print(data.keys())
        link_capacity_orig = data['link_capacity'].numpy()
        flow_traffic = (data['flow_traffic'].numpy() - normalizers["flow_traffic"][0]) *normalizers["flow_traffic"][1]
        flow_packets = (data['flow_packets'].numpy() - normalizers["flow_packets"][0]) *normalizers["flow_packets"][1]
        link_capacity = (data['link_capacity'].numpy()  - normalizers["link_capacity"][0]) *normalizers["link_capacity"][1]
        flow_packet_size = (data['flow_packet_size'].numpy() - normalizers["flow_packet_size"][0])  * normalizers["link_capacity"][1]
        flow_type = data['flow_type'].numpy()
        flow_length = data['flow_length'].numpy()
        link_to_path = data['link_to_path'].numpy()
        flow_ipg_mean = data['flow_ipg_mean'].numpy()
        flow_ipg_var = data['flow_ipg_var'].numpy()
        path_to_link = data['path_to_link'].numpy()
        y_vals = torch.tensor(labels)








        all_flows_used = torch.tensor(np.concatenate([flow_traffic,flow_packets, flow_type, flow_type, np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )

        outputs = model(link_to_path, path_to_link, link_capacity,all_flows_used, flow_traffic, link_capacity_orig)

        predictions.extend(outputs.detach().numpy())
        loss += criterion(outputs, y_vals ) # Compute the loss.
        if i%fake_batch_size == 0 or i== len(train_dataset):
          loss = loss/fake_batch_size
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          optimizer.zero_grad()  # Clear gradients.
          loss = torch.tensor([0.0])

        pbar.set_description(f"Loss: {loss.item()/fake_batch_size:.4f}")
        y_all.extend(labels)
    print("train loss = ",criterion(torch.tensor(predictions), torch.tensor(y_all )).item())


    model.eval()
    with tqdm(val_dataset, total=len(val_dataset)) as pbar:
      predictions = []
      y_all = []
      for batch in  pbar:
          # print(batch)
          # Process the batch as needed
          data, labels = batch  # Modify this according to the structure of your data
          # print(data.keys())
          link_capacity_orig = data['link_capacity'].numpy()
          flow_traffic = (data['flow_traffic'].numpy() - normalizers["flow_traffic"][0]) *normalizers["flow_traffic"][1]
          flow_packets = (data['flow_packets'].numpy() - normalizers["flow_packets"][0]) *normalizers["flow_packets"][1]
          link_capacity = (data['link_capacity'].numpy()  - normalizers["link_capacity"][0]) *normalizers["link_capacity"][1]
          flow_packet_size = (data['flow_packet_size'].numpy() - normalizers["flow_packet_size"][0])  * normalizers["link_capacity"][1]
          flow_type = data['flow_type'].numpy()
          flow_length = data['flow_length'].numpy()
          link_to_path = data['link_to_path'].numpy()
          flow_ipg_mean = data['flow_ipg_mean'].numpy()
          flow_ipg_var = data['flow_ipg_var'].numpy()
          path_to_link = data['path_to_link'].numpy()
          # y_vals = torch.tensor(labels)

          # path_gather_traffic = 0
          # load = torch.sum(path_gather_traffic, dim=1) / (link_capacity * 1e9)
          all_flows_used = torch.tensor(np.concatenate([flow_traffic,flow_packets, flow_type, flow_type, np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )

          outputs = model(link_to_path, path_to_link, link_capacity,all_flows_used, flow_traffic, link_capacity_orig)
          predictions.extend(outputs.detach().numpy())
          y_all.extend(labels)

          # scheduler.step(criterion(torch.tensor(predictions), torch.tensor(y_all )))
    print("val loss = ",criterion(torch.tensor(predictions), torch.tensor(y_all )).item())


In [ ]:
for epoch in range(1, 150):
    print("epoch ", epoch)
    train()

epoch  1


Loss: 0.3750: 100%|██████████| 3380/3380 [20:02<00:00,  2.81it/s]


train loss =  1.8529808521270752


100%|██████████| 845/845 [02:15<00:00,  6.24it/s]


val loss =  1.0
epoch  2


...

Loss: 0.3750: 100%|██████████| 3380/3380 [19:13<00:00,  2.93it/s]


train loss =  1.0


100%|██████████| 845/845 [02:17<00:00,  6.16it/s]


val loss =  1.0
epoch  5


Loss: 0.8750:  73%|███████▎  | 2456/3380 [13:38<02:28,  6.21it/s]

In [ ]:

class MyCombinedModel(nn.Module):
    def __init__(self, link_state_dim, path_state_dim, num_layers=2):
        super(MyCombinedModel, self).__init__()
        self.link_embed = LinkEmbedding(link_state_dim)  # Assuming LinkEmbedding is defined
        self.flow_embedded = PathEmbedding(path_state_dim)  # Assuming PathEmbedding is defined
        self.reader = PathReadout(path_state_dim, link_state_dim)  # Assuming PathReadout is defined
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=link_state_dim, dim_feedforward=64, batch_first=True, nhead=2)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.link_update = nn.GRUCell(
          link_state_dim, link_state_dim
        )
    def forward(self, link_to_path, path_to_link,  link_capacity,  all_flows_used, flow_traffic, link_capacity_orig):

        flow_embedding = self.flow_embedded(all_flows_used)





        all_link_capacities_used = []
        loads = []

        for i in range(len(path_to_link)):

            loads.append(torch.sum(torch.index_select(torch.tensor(flow_traffic), 0, torch.tensor(path_to_link[i][:, 0 ]).type(torch.int64)))/  (link_capacity_orig[i] * 1e9))


        loads_capacities = torch.cat((torch.tensor(loads).unsqueeze(1), torch.tensor(link_capacity)), dim=1)

        capacity_gather = []
        for i, values in enumerate(link_to_path):
          capacity_gather.append(link_capacity_orig[values])
          all_link_capacities_used.append(self.link_embed(torch.tensor(loads_capacities)[values]))

        prev_path_state = all_link_capacities_used
        print("len of link_to_path = ", len(link_to_path), "path to link ",  len(path_to_link), "len(all_link_capacities_used) ", len(all_link_capacities_used))
        for _ in range(2):
          print(_, flow_embedding.unsqueeze(1).shape, torch.cat(all_link_capacities_used, dim=0).shape , torch.nn.utils.rnn.pad_sequence(all_link_capacities_used, batch_first =True).shape)
          input_sequence = torch.cat((flow_embedding.unsqueeze(1),torch.nn.utils.rnn.pad_sequence(all_link_capacities_used, batch_first =True)), dim=1)
          encoder_out = self.transformer_encoder(input_sequence)
          flow_embedding = encoder_out[:, -1, :]

          all_link_capacities_used = []
          for i, path_link in enumerate(path_to_link):
              link_state = self.link_update(torch.index_select(encoder_out[torch.tensor(path_link[:, 0])], 1, torch.tensor(path_link[:, 1]) ).sum(axis=1), )
              # print(link_state.shape)
              all_link_capacities_used.append(link_state)



        output = self.reader(encoder_out[:, 1:, :])

        delays = []

        for i, out in enumerate(output.squeeze()):
            delays.append(torch.sum(out[:len(capacity_gather[i])]/torch.tensor(capacity_gather[i]).T[0]).unsqueeze(0))


        return torch.cat(delays)

link_state_dim = 64
path_state_dim = 64

from torch.optim.lr_scheduler import ReduceLROnPlateau
num_layers = 2  # Example value for the number of layers

model = MyCombinedModel(link_state_dim,  path_state_dim, num_layers)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer =  torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
# scheduler = ReduceLROnPlateau(optimizer, patience=10, mode= 'min')
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.00001, max_lr=0.1)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = MeanAbsolutePercentageError()

def train():
  model.train()
  fake_batch_size= 32
  with tqdm(train_dataset, total=len(train_dataset)) as pbar:
    predictions = []
    y_all = []
    loss = 0
    loss = torch.tensor([0.0])

    for i,batch in enumerate(pbar):
        # print(batch)
        # Process the batch as needed
        data, labels = batch  # Modify this according to the structure of your data
        # print(data.keys())
        link_capacity_orig = data['link_capacity'].numpy()
        flow_traffic = (data['flow_traffic'].numpy() - normalizers["flow_traffic"][0]) *normalizers["flow_traffic"][1]
        flow_packets = (data['flow_packets'].numpy() - normalizers["flow_packets"][0]) *normalizers["flow_packets"][1]
        link_capacity = (data['link_capacity'].numpy()  - normalizers["link_capacity"][0]) *normalizers["link_capacity"][1]
        flow_packet_size = (data['flow_packet_size'].numpy() - normalizers["flow_packet_size"][0])  * normalizers["link_capacity"][1]
        flow_type = data['flow_type'].numpy()
        flow_length = data['flow_length'].numpy()
        link_to_path = data['link_to_path'].numpy()
        flow_ipg_mean = data['flow_ipg_mean'].numpy()
        flow_ipg_var = data['flow_ipg_var'].numpy()
        path_to_link = data['path_to_link'].numpy()
        y_vals = torch.tensor(labels)








        all_flows_used = torch.tensor(np.concatenate([flow_traffic,flow_packets, flow_type, flow_type, np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )

        outputs = model(link_to_path, path_to_link, link_capacity,all_flows_used, flow_traffic, link_capacity_orig)

        predictions.extend(outputs.detach().numpy())
        loss += criterion(outputs, y_vals ) # Compute the loss.
        if i%fake_batch_size == 0 or i== len(train_dataset):
          loss = loss/fake_batch_size
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          optimizer.zero_grad()  # Clear gradients.
          loss = torch.tensor([0.0])

        pbar.set_description(f"Loss: {loss.item()/fake_batch_size:.4f}")
        y_all.extend(labels)
    print("train loss = ",criterion(torch.tensor(predictions), torch.tensor(y_all )).item())


    model.eval()
    with tqdm(val_dataset, total=len(val_dataset)) as pbar:
      predictions = []
      y_all = []
      for batch in  pbar:
          # print(batch)
          # Process the batch as needed
          data, labels = batch  # Modify this according to the structure of your data
          # print(data.keys())
          link_capacity_orig = data['link_capacity'].numpy()
          flow_traffic = (data['flow_traffic'].numpy() - normalizers["flow_traffic"][0]) *normalizers["flow_traffic"][1]
          flow_packets = (data['flow_packets'].numpy() - normalizers["flow_packets"][0]) *normalizers["flow_packets"][1]
          link_capacity = (data['link_capacity'].numpy()  - normalizers["link_capacity"][0]) *normalizers["link_capacity"][1]
          flow_packet_size = (data['flow_packet_size'].numpy() - normalizers["flow_packet_size"][0])  * normalizers["link_capacity"][1]
          flow_type = data['flow_type'].numpy()
          flow_length = data['flow_length'].numpy()
          link_to_path = data['link_to_path'].numpy()
          flow_ipg_mean = data['flow_ipg_mean'].numpy()
          flow_ipg_var = data['flow_ipg_var'].numpy()
          path_to_link = data['path_to_link'].numpy()
          # y_vals = torch.tensor(labels)

          # path_gather_traffic = 0
          # load = torch.sum(path_gather_traffic, dim=1) / (link_capacity * 1e9)
          all_flows_used = torch.tensor(np.concatenate([flow_traffic,flow_packets, flow_type, flow_type, np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )

          outputs = model(link_to_path, path_to_link, link_capacity,all_flows_used, flow_traffic, link_capacity_orig)
          predictions.extend(outputs.detach().numpy())
          y_all.extend(labels)

          # scheduler.step(criterion(torch.tensor(predictions), torch.tensor(y_all )))
    print("val loss = ",criterion(torch.tensor(predictions), torch.tensor(y_all )).item())

      # link_gather = torch.gather(torch.tensor(link_capacity), 1,tensor_array, sparse_grad =True, )

      # print(link_gather)



train()

In [ ]:
for epoch in range(1, 50):
    print("epoch ", epoch)
    train()
    # train_acc = test(custom_dataset)
    # test_acc = test(test_loader)
    # print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    # print(f'Epoch: {epoch:03d}, MAPE: {train_acc:.4f}')


model_path = "model.pth"

# Save the model
torch.save(model.state_dict(), model_path)


In [ ]:
!pip install torchrec


In [ ]:
# Define the ragged array
ragged_array = [[29, 1, 37, 9, 42, 10, 48],
                [28, 29, 1, 37, 8, 45, 13, 49, 27]]

# Define the 1D tensor
tensor_1d = torch.arange(50)  # Example 1D tensor

# Convert the ragged array to a 2D PyTorch tensor
padded_array = [torch.LongTensor(sublist) for sublist in ragged_array]
max_length = max(len(sublist) for sublist in ragged_array)
padded_array = [torch.cat((t, torch.zeros(max_length - len(t), dtype=torch.long))) for t in padded_array]
tensor_array = torch.stack(padded_array)

# Use the 2D tensor to index the 1D tensor
result = tensor_1d[tensor_array]

# Filter out the zeros
result_ragged = [row[row != 0].tolist() for row in result]

# Print the result as a ragged array
print(result_ragged)

In [ ]:
link_state_dim = 32
path_state_dim = 32 # Example value for link_state_dim

# Generate an example input tensor
input_tensor = torch.randn(72, 7)  # Example input tensor of shape 72x7
print(input_tensor.shape)
# Perform the forward pass
output = model_embedded(input_tensor)
print(output.shape)  # Check the shape of the output